In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Import library
import glob
import pandas as pd
import os
import shutil
!pwd
%cd /content/drive/My Drive/Colab Notebooks

Mounted at /content/drive
/content
/content/drive/My Drive/Colab Notebooks


In [3]:
df_demand = pd.read_excel('Use case/Data.xlsx', skiprows=1, usecols=['Job start date','Job open Qty',"Job Item",'On Hand Qty','On Hold Qty'],sheet_name ='FNDWRR' ).fillna(0)
df_demand.head()

,Job Item,Job open Qty,On Hold Qty,On Hand Qty,Job start date
0,1998082,390.0,0.0,7807.0,2022-04-01
1,1998083,0.0,0.0,895.0,2022-03-08
2,1998083,0.0,0.0,895.0,2022-03-08
3,1998083,0.0,0.0,895.0,2022-03-09
4,1998083,0.0,0.0,895.0,2022-03-09


In [4]:
df_supply = pd.read_excel('Use case/Data.xlsx', skiprows=6,sheet_name ='supply')
df_supply.head()

,Part_No,2022-03-14 00:00:00,2022-03-21 00:00:00,2022-03-28 00:00:00,2022-04-04 00:00:00,2022-04-11 00:00:00,2022-04-18 00:00:00,2022-04-25 00:00:00,2022-05-02 00:00:00,2022-05-09 00:00:00,2022-05-16 00:00:00,2022-05-23 00:00:00,2022-05-30 00:00:00,2022-06-06 00:00:00,2022-06-13 00:00:00,2022-06-20 00:00:00,2022-06-27 00:00:00,2022-07-04 00:00:00,2022-07-11 00:00:00
0,140331037,NaN,NaN,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,140340037,NaN,49377.0,23641.0,67463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,140341001,NaN,NaN,NaN,1250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,140341028,1762.0,10504.0,33018.0,1452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,208998040,NaN,6800.0,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# List columns of Job item, Onhand stock v& On Hold Qty
list_stock_cols = ['Job Item','On Hand Qty','On Hold Qty']

# Remove duplicates
df_on_hand_stock = df_demand.drop_duplicates(subset=list_stock_cols)[list_stock_cols]
df_on_hand_stock['On Hand Total']= df_on_hand_stock['On Hand Qty']+ df_on_hand_stock['On Hold Qty']
df_on_hand_stock.drop(columns=['On Hand Qty','On Hold Qty'],inplace = True)
df_on_hand_stock

,Job Item,On Hand Total
0,1998082,7807.0
1,1998083,895.0
166,1998163,0.0
201,1998164,0.0
202,1998165,0.0
...,...,...
32904,001998185DG9C,8421.0
32912,001998339DG9,949.0
32915,001998339DG9C,904.0
32918,001998339DG9S,0.0


In [6]:
#Rename column Part_no in sheet supply 
df_supply.rename(columns={"Part_No     ": "Job Item"}, inplace= True)

In [7]:
#Get on-transit qty
df_supply_melted = df_supply.melt(id_vars=['Job Item'],var_name='supply_week',value_name='Planning OH qty')
df_supply_melted.head()

,Job Item,supply_week,Planning OH qty
0,140331037,2022-03-14,NaN
1,140340037,2022-03-14,NaN
2,140341001,2022-03-14,NaN
3,140341028,2022-03-14,1762.0
4,208998040,2022-03-14,NaN


In [8]:
#Insert 1 time column into "on hand" sheet with value 2000 (regard "on hand" was available a long time ago) 
df_on_hand_stock.insert(1,'supply_week', pd.to_datetime('2000-01-01'))
df_on_hand_stock.columns = df_supply_melted.columns
df_on_hand_stock.head(2)

,Job Item,supply_week,Planning OH qty
0,1998082,2000-01-01,7807.0
1,1998083,2000-01-01,895.0


In [9]:
# Merge stock-on-hand and on-transit into 1 for supply sheet 
df_supply_all = pd.concat([df_on_hand_stock,df_supply_melted])
df_supply_all.head()

,Job Item,supply_week,Planning OH qty
0,1998082,2000-01-01,7807.0
1,1998083,2000-01-01,895.0
166,1998163,2000-01-01,0.0
201,1998164,2000-01-01,0.0
202,1998165,2000-01-01,0.0


In [10]:
# Groupby by Job item and calculate cumsum Supply Qty
df_supply_all['cumsum_supply'] = df_supply_all.groupby('Job Item').cumsum()
df_supply_all.head()

,Job Item,supply_week,Planning OH qty,cumsum_supply
0,1998082,2000-01-01,7807.0,7807.0
1,1998083,2000-01-01,895.0,895.0
166,1998163,2000-01-01,0.0,0.0
201,1998164,2000-01-01,0.0,0.0
202,1998165,2000-01-01,0.0,0.0


In [11]:
# Remove any Job Item has demand = 0
df_real_demand = df_demand[df_demand['Job open Qty'] != 0].drop(columns=['On Hand Qty','On Hold Qty'])

# Groupby by Job item and calculate cumsum Demand Qty
df_real_demand['cumsum_demand'] = df_real_demand.groupby('Job Item').cumsum()
df_real_demand

,Job Item,Job open Qty,Job start date,cumsum_demand
0,1998082,390.0,2022-04-01,390.0
42,1998083,572.0,2022-03-18,572.0
248,1998553,10.0,2022-04-01,10.0
249,103000085,332.0,2022-03-23,332.0
250,103323003,1100.0,2022-03-15,1100.0
...,...,...,...,...
32912,001998339DG9,580.0,2022-03-16,580.0
32916,001998339DG9C,436.0,2022-03-21,436.0
32917,001998339DG9C,360.0,2022-03-31,796.0
32921,001998656C,100.0,2022-03-30,100.0


In [12]:
# Merge demand and supply sheet
df_demand_supply = df_real_demand.merge(df_supply_all,how='right', on='Job Item')
df_demand_supply.head()

,Job Item,Job open Qty,Job start date,cumsum_demand,supply_week,Planning OH qty,cumsum_supply
0,1998082,390.0,2022-04-01,390.0,2000-01-01,7807.0,7807.0
1,1998083,572.0,2022-03-18,572.0,2000-01-01,895.0,895.0
2,1998163,NaN,NaT,NaN,2000-01-01,0.0,0.0
3,1998164,NaN,NaT,NaN,2000-01-01,0.0,0.0
4,1998165,NaN,NaT,NaN,2000-01-01,0.0,0.0


In [13]:
df_supply_all[df_supply_all['Job Item'] == 103323003]

,Job Item,supply_week,Planning OH qty,cumsum_supply
250,103323003,2000-01-01,8002.0,8002.0


In [14]:
df_demand_supply[df_demand_supply['Job Item'] == 103323003]

,Job Item,Job open Qty,Job start date,cumsum_demand,supply_week,Planning OH qty,cumsum_supply
8,103323003,1100.0,2022-03-15,1100.0,2000-01-01,8002.0,8002.0
9,103323003,1200.0,2022-03-16,2300.0,2000-01-01,8002.0,8002.0
10,103323003,1730.0,2022-03-30,4030.0,2000-01-01,8002.0,8002.0
11,103323003,1238.0,2022-03-31,5268.0,2000-01-01,8002.0,8002.0
12,103323003,1730.0,2022-03-31,6998.0,2000-01-01,8002.0,8002.0
13,103323003,1000.0,2022-04-09,7998.0,2000-01-01,8002.0,8002.0


In [15]:
# Remove rows that had insufficent supply for demand
df_enough = df_demand_supply[df_demand_supply['cumsum_demand'] <= df_demand_supply['cumsum_supply']]

In [65]:
# Get only the first row of sufficent supply 
df_first_supply = df_enough.groupby(['Job Item','cumsum_demand']).first()
df_first_supply.head()

Job open Qty Job start date supply_week  \
Job Item  cumsum_demand                                            
1998082   390.0                 390.0     2022-04-01  2000-01-01   
1998083   572.0                 572.0     2022-03-18  2000-01-01   
1998553   10.0                   10.0     2022-04-01  2000-01-01   
103323003 1100.0               1100.0     2022-03-15  2000-01-01   
          2300.0               1200.0     2022-03-16  2000-01-01   

                         Planning OH qty  cumsum_supply  
Job Item  cumsum_demand                                  
1998082   390.0                   7807.0         7807.0  
1998083   572.0                    895.0          895.0  
1998553   10.0                      12.0           12.0  
103323003 1100.0                  8002.0         8002.0  
          2300.0                  8002.0         8002.0

In [66]:
# Retain necessary rows
df_first_supply = df_first_supply.reset_index()[['Job Item','Job open Qty','Job start date','cumsum_demand','cumsum_supply','supply_week','Planning OH qty']]
df_first_supply.head()

,Job Item,Job open Qty,Job start date,cumsum_demand,cumsum_supply,supply_week,Planning OH qty
0,1998082,390.0,2022-04-01,390.0,7807.0,2000-01-01,7807.0
1,1998083,572.0,2022-03-18,572.0,895.0,2000-01-01,895.0
2,1998553,10.0,2022-04-01,10.0,12.0,2000-01-01,12.0
3,103323003,1100.0,2022-03-15,1100.0,8002.0,2000-01-01,8002.0
4,103323003,1200.0,2022-03-16,2300.0,8002.0,2000-01-01,8002.0


In [69]:
# Label rows having insufficent supply
df_final = df_real_demand.merge(df_first_supply,how='left')

# Format 'Job start date' into datetime format
df_final['Job start date'] = pd.to_datetime(df_final['Job start date'])#.dt.strftime('%d-%m-%Y')

# Calculate late days
df_final['Late (day>0)']=(df_final['supply_week']-df_final['Job start date']).dt.days

# Note for insufficient supply or late supply
df_final.loc[df_final['Late (day>0)'] > 0,['Note']]='Tre hang'
df_final.loc[df_final['cumsum_supply'].isnull(),['Note']]='Khong co hang'

#Format 'Job start date' and 'supply_week' into day-montnh-year
df_final['supply_week']= df_final['supply_week'].dt.strftime('%d-%m-%Y')
df_final['Job start date']= df_final['Job start date'].dt.strftime('%d-%m-%Y')
df_final.head()

,Job Item,Job open Qty,Job start date,cumsum_demand,cumsum_supply,supply_week,Planning OH qty,Late (day>0),Note
370,280160805,2000.0,21-03-2022,2000.0,10224.0,14-03-2022,10151.0,-7.0,NaN
371,280160805,2000.0,21-03-2022,4000.0,10224.0,14-03-2022,10151.0,-7.0,NaN
372,280160805,2000.0,21-03-2022,6000.0,10224.0,14-03-2022,10151.0,-7.0,NaN
373,280160805,2000.0,22-03-2022,8000.0,10224.0,14-03-2022,10151.0,-8.0,NaN
374,280160805,2000.0,22-03-2022,10000.0,10224.0,14-03-2022,10151.0,-8.0,NaN
375,280160805,2000.0,22-03-2022,12000.0,31224.0,21-03-2022,21000.0,-1.0,NaN
376,280160805,2000.0,23-03-2022,14000.0,31224.0,21-03-2022,21000.0,-2.0,NaN
377,280160805,2000.0,23-03-2022,16000.0,31224.0,21-03-2022,21000.0,-2.0,NaN
378,280160805,2000.0,24-03-2022,18000.0,31224.0,21-03-2022,21000.0,-3.0,NaN
379,280160805,2000.0,24-03-2022,20000.0,31224.0,21-03-2022,21000.0,-3.0,NaN


In [70]:
# Format color 
def highlight_rows(row):
    val = row.loc['Note']
    if val == 'Tre hang':
       color = '#FFB3BA'
    elif val == 'Khong co hang':
       color =  '#BAFFC9'
    else: color =  None
    return ['background-color: {}'.format(color) for r in row]

df_format = df_final.style.apply(highlight_rows,axis=1)

In [71]:
# Xuất file excel
df_format.to_excel('Use case/Output data.xlsx')